## Task 1: Linear Regression


All the necessary imports. You are not allowed to import any additional functionality for this task.

In [9]:
import os
import numpy as np
import matplotlib.pyplot as plt

Please fill in all the ToDos.

In [10]:
def f(x):
    return x ** 3 + (x - .5) ** 2 + .5


def sample_data(n, low, high, sigma):
    x = np.random.rand(int(n)) * (high - low) + low
    return x.reshape(-1, 1), f(x) + np.random.randn(int(n)) * sigma


def visualize(X_train, y_train,
              X_test=None, y_test=None,
              X_val=None, y_val=None,
              X_pred=None, y_pred=None, y_std_pred=None,
              filename=None):
    plt.plot(X_train, y_train, '.', color='k')
    if X_test is not None and y_test is not None:
        plt.plot(X_test, y_test, 'x', color='r')

    if X_val is not None and y_val is not None:
        plt.plot(X_val, y_val, '.', color='tab:orange')

    if y_std_pred is not None:
        for c in [1., 2., 3.]:
            plt.fill_between(X_pred.squeeze(), y_pred - c * y_std_pred, y_pred + c * y_std_pred,
                             color=(0, 0, 1, 0.05))

    if X_pred is not None and y_pred is not None:
        plt.plot(X_pred.squeeze(), y_pred, '-', color='b')

    if filename:
        plt.savefig(filename)
    else:
        plt.show()
    plt.close()


def RMSE(pred, gt):
  #TASK 1a)
  #ToDo: Compute Root Mean Square Error
  RMSE = np.sqrt(np.mean((pred - gt) ** 2))
  return RMSE


def log_likelihood(mean, std, gt):
  #ToDo: Compute the log-likelihood
  var = std ** 2
  log_likelihood = -0.5 * np.log(2 * np.pi * var) - 0.5 * ((gt - mean) ** 2) / var
  return np.mean(log_likelihood)


def add_bias(X):
  #ToDo
  #TASK 1a)
  X_with_Bias = np.hstack([X, np.ones((X.shape[0], 1))])
  return X_with_Bias


def polynomial_features(X, degree):
  #ToDo: Initalize the features array with the correct dimensions
  n_samples = X.shape[0]
  features = np.zeros((n_samples, degree))

  #ToDo: Compute the polynomial representation of the given features
  for i in range(degree):
      features[:, i] = X[:, 0] ** (i + 1)
  return features


def squared_exponential_features(X, alpha, beta):
  #ToDo: Initalize the features array with the correct dimensions

  n = X.shape[0]
  k = len(alpha)
  features = np.zeros((n, k))
  for i in range(n):
      for j in range(k):
          features[i, j] = np.exp(-0.5 * beta * (X[i, 0] - alpha[j]) ** 2)
  return features


class LinearRegression:
    def __init__(self):
        self.bias = None
        self.w = None

    def fit(self, X, y, lam, bias=True):
        #TASK 1a)
        #ToDo: Set bias if bias is present
        self.bias = bias

        #Todo: Add the bias to X
        if self.bias:
            X = add_bias(X)

        #Todo: Fit the weights
        I = np.eye(X.shape[1])
        self.w = np.linalg.inv(X.T @ X + lam * I) @ X.T @ y

    def predict(self, X_):
        #TASK 1a)
        #ToDo: Compute the prediciton
        if self.bias:
            X_ = add_bias(X_)

        prediction = X_ @ self.w
        return prediction


class BayesianLinearRegression:
    def __init__(self):
        self.bias = None
        self.lam = None
        self.sigma = None
        self.X = None
        self.y = None
        self.mu = None
        self.Lambda = None
        self.Lambda_inv = None

    def fit(self, X, y, lam, sigma, bias=True):
      #ToDo: Initialize the relevant attributes with the parameters
        self.bias = bias
        self.lam = lam
        self.sigma = sigma

        #ToDo: Add bias to X
        if self.bias:
            X = add_bias(X)

        #Todo: Compute these values
        n, d = X.shape
        I = np.eye(d)

        self.Lambda = lam * I + (1 / sigma**2) * X.T @ X
        self.Lambda_inv = np.linalg.inv(self.Lambda)
        self.mu = (1 / sigma**2) * self.Lambda_inv @ X.T @ y
        self.X = X
        self.y = y

    def predict(self, X_):
      #ToDo: Add bias to X
        if self.bias:
            X_ = add_bias(X_)

        #ToDo: Compute these values
        mean = X_ @ self.mu
        std = np.sqrt(np.sum(X_ @ self.Lambda_inv * X_, axis=1)) * self.sigma
        return mean, std

    def log_marginal_likelihood(self):
        n, k = self.X.shape
        log_marginal = .5 * k * np.log(self.lam)
        log_marginal -= n * np.log(self.sigma)
        log_marginal -= .5 * np.sum((self.y - self.X @ self.mu) ** 2) / (self.sigma ** 2)
        log_marginal += .5 * self.lam * self.mu.T @ self.mu
        log_marginal -= .5 * np.linalg.slogdet(self.Lambda)[1]
        log_marginal -= .5 * n * np.log(2 * np.pi)
        return log_marginal

Run this cell to execute your code and to generate all the plots. Don't forget to upload both datasets lin_reg_train.txt and lin_reg_test.txt.

In [11]:
if __name__ == '__main__':
    sigma = 0.1
    lam = 0.01

    # load data
    print(os.path.join(os.getcwd(), "lin_reg_test.txt"))
    train = np.loadtxt(os.path.join(os.getcwd(), "lin_reg_train.txt"), delimiter=" ", encoding="utf-8-sig")
    test = np.loadtxt(os.path.join(os.getcwd(), "lin_reg_test.txt"), delimiter=" ", encoding="utf-8-sig")
    X_train = train[:, 0].reshape(-1, 1)
    y_train = train[:, 1]
    X_test = test[:, 0].reshape(-1, 1)
    y_test = test[:, 1]
    # for plots
    X_pred = np.linspace(-1, 1, 100).reshape(-1, 1)
    visualize(X_train, y_train, X_test=X_test, y_test=y_test,
              filename=os.path.join(os.getcwd(), 'data.pdf'))

    n_train = X_train.shape[0]
    n_test = X_test.shape[0]

    print("Linear Features")
    model = LinearRegression()
    model.fit(X_train, y_train, lam, bias=True)
    print("  Train RMSE: {}".format(RMSE(model.predict(X_train), y_train)))
    print("  Test  RMSE: {}".format(RMSE(model.predict(X_test), y_test)))
    print()
    y_pred = model.predict(X_pred)
    visualize(X_train, y_train, X_pred=X_pred, y_pred=y_pred,
              filename=os.path.join(os.getcwd(), 'linear.pdf'))

    print("Polynomial Features")
    degrees = [2, 3, 4]
    for degree in degrees:
        print("  Degree = {}".format(degree))
        P_train = polynomial_features(X_train, degree)
        P_test = polynomial_features(X_test, degree)
        P_pred = polynomial_features(X_pred, degree)
        model = LinearRegression()
        model.fit(P_train, y_train, lam, bias=True)
        print("  Train RMSE: {}".format(RMSE(model.predict(P_train), y_train)))
        print("  Test  RMSE: {}".format(RMSE(model.predict(P_test), y_test)))
        print()
        y_pred = model.predict(P_pred)
        visualize(X_train, y_train, X_pred=X_pred, y_pred=y_pred,
                  filename=os.path.join(os.getcwd(), 'polynomial_{}.pdf'.format(degree)))

    print("Bayesian Linear Regression")
    model = BayesianLinearRegression()
    model.fit(X_train, y_train, lam, sigma, bias=True)
    mean_pred_train, std_pred_train = model.predict(X_train)
    mean_pred_test, std_pred_test = model.predict(X_test)
    print("  Train RMSE: {}".format(RMSE(mean_pred_train, y_train)))
    print("  Test  RMSE: {}".format(RMSE(mean_pred_test, y_test)))
    print("  Average Train Log-LLH: {}".format(log_likelihood(mean_pred_train, std_pred_train, y_train)))
    print("  Average Test  Log-LLH: {}".format(log_likelihood(mean_pred_test, std_pred_test, y_test)))
    print()
    y_pred, y_std_pred = model.predict(X_pred)
    visualize(X_train, y_train, X_pred=X_pred, y_pred=y_pred, y_std_pred=y_std_pred,
              filename=os.path.join(os.getcwd(), 'bayesian_linear.pdf'))

    print("Squared Exponential Features")
    model = BayesianLinearRegression()
    alpha = np.linspace(-0.9, 1, 20)
    beta = 10.
    SE_train = squared_exponential_features(X_train, alpha, beta)
    SE_test = squared_exponential_features(X_test, alpha, beta)
    SE_pred = squared_exponential_features(X_pred, alpha, beta)
    model.fit(SE_train, y_train, lam, sigma, bias=True)
    mean_pred_train, std_pred_train = model.predict(SE_train)
    mean_pred_test, std_pred_test = model.predict(SE_test)
    print("  Train RMSE: {}".format(RMSE(mean_pred_train, y_train)))
    print("  Test  RMSE: {}".format(RMSE(mean_pred_test, y_test)))
    print("  Average Train Log-LLH: {}".format(log_likelihood(mean_pred_train, std_pred_train, y_train)))
    print("  Average Test  Log-LLH: {}".format(log_likelihood(mean_pred_test, std_pred_test, y_test)))
    print()
    y_pred, y_std_pred = model.predict(SE_pred)
    visualize(X_train, y_train, X_pred=X_pred, y_pred=y_pred, y_std_pred=y_std_pred,
              filename=os.path.join(os.getcwd(), 'bayesian_squared_exponential.pdf'))

/Users/masprime77/Documents/TUDa/4_semester_SoSe25/SML/Homeworks/Homework_03/Data/lin_reg_test.txt
Linear Features
  Train RMSE: 0.4121780156736108
  Test  RMSE: 0.38428816992597875

Polynomial Features
  Degree = 2
  Train RMSE: 0.21201447265968612
  Test  RMSE: 0.2168724271414873

  Degree = 3
  Train RMSE: 0.0870682129548175
  Test  RMSE: 0.1083580371973804

  Degree = 4
  Train RMSE: 0.0870126130663818
  Test  RMSE: 0.10666239820964406

Bayesian Linear Regression
  Train RMSE: 0.41217792591659724
  Test  RMSE: 0.38434085452132954
  Average Train Log-LLH: -27568.07867786533
  Average Test  Log-LLH: -25265.407986599534

Squared Exponential Features
  Train RMSE: 0.0816094153080289
  Test  RMSE: 0.14341009678374458
  Average Train Log-LLH: -244.12272537111875
  Average Test  Log-LLH: -225.88273995771985



## Task 2: Gaussian Process

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def rbf_kernel(X1, X2, rbf_sigma=1):
  """
  Compute the covariance matrix between sets of 1D points X1 and X2 using the RBF/Gaussian kernel.

  Args:
    X1: ndarray(m,)
    X2: ndarray(n,)
    rbf_sigma: kernel bandwidth (float)

  Returns:
    cov: ndarray(m, n)
  """
  ## TODO: your code here
  pass


In [ ]:
## RBF kernel test
test_X1 = np.asarray([5, 9, 6])
test_X2 = np.asarray([5, 9, 3, 4, 1])
rbf_gt = [[1.0, 0.00033546262790251185, 0.1353352832366127, 0.6065306597126334, 0.00033546262790251185], [0.00033546262790251185, 1.0, 1.522997974471263e-08, 3.726653172078671e-06, 1.2664165549094176e-14], [0.6065306597126334, 0.011108996538242306, 0.011108996538242306, 0.1353352832366127, 3.726653172078671e-06]]
rbf_gt = np.asarray(rbf_gt)

print("RBF ground truth")
print(rbf_gt)

print("RBF test")
print(rbf_kernel(test_X1, test_X2, rbf_sigma=1))



In [ ]:
np.random.seed(42) # fixed seed, do not change
x = np.arange(0, 2*np.pi, 0.015) # test points
## TODO: complete the ...
y = ... # Target function
s2 = ... # noise variance
noise = ...
t = y + noise # Observed target function

In [ ]:
K = rbf_kernel(x, x) # squared exponential kernel

# Intialize the GP mean and covariance with the correct shape
m = np.zeros(x.shape) # GP mean
S = K + s2*np.eye(K.shape[0]) # GP covariance

# Initial GP without any training points
plt.figure()
plt.title('Number of observations: 0', fontsize=18)
plt.grid('on')
plt.plot(x, y, lw=2, color='blue')
plt.plot(x, m, lw=2, color='red')
plt.fill_between(x, m+2*np.sqrt(np.diag(S)), m-2*np.sqrt(np.diag(S)), facecolor='red', alpha=0.5)

In [ ]:
# At each iteration, sample the test point with the highest uncertainty as the new training point and update your GP mean and covariance.
# Plot GP at iteration 1, 2, 5, 10, 15. The iteration should be equal to the number of sampled points, e.g. at iteration 5 there are 5 points sampled.
# TODO: your code here

## Task 6: Principal Component Analysis

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
data = np.loadtxt(os.path.join(os.getcwd(), 'iris.txt'), delimiter=',', encoding='utf-8-sig')
n = data.shape[0]
X = data[:,:-1] # last column denotes the class, so we ignore it

#ToDo: normalize data
X_mean = ...
X_std = ...
X_norm = ...

In [ ]:
#ToDo: Compute Covariance matrix
A = ...

#ToDo: Compute Eigenvalues and Eigenvectors Hint: Check if Numpy has a function for this
eigval, eigvec = ...

idx = np.argsort(-eigval)
eigval = eigval[idx]
eigvec = eigvec[:,idx]

#ToDo: Compute the variance explained by each eigenvalue Hint: Check if Numpy has a function for this
explained_variance = ...

#Hint: The resulting plot should resemble an elbow
plt.figure(figsize=(8,6))
plt.plot([1,2,3,4],explained_variance)
plt.xlabel('N. of components', fontsize=18)
plt.ylabel('Explained variance', fontsize=18)
plt.grid(True)

In [ ]:
#ToDo Compute the projection
X_proj = ...

plt.figure(figsize=(8,6))

#Todo: Use the projected data to plot the 3 classes as scatter plots where each class has a different colour.


In [ ]:
#ToDo: Initalize the normalized RMSE arrays with the correct dimensions
nrmse = ...


for i in range(0,...):
  #ToDo: Compute the Projection of the data
  X_proj = ...
  #ToDo: Project the resulting data back
  X_restored = ...
  X_restored = ...


  nrmse[i,:] = ...


print(nrmse)
